In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("./Data_Processed_All_Contractors_Characteristics - Data.csv")

In [3]:
df.shape

(5046, 71)

In [4]:
def find_index(name):
    index_num = columns_list.index(name)
    print(index_num)

In [5]:
df_2016 = df.loc[df['registed 2016'] == 1]

In [6]:
df_2016.shape

(1473, 71)

In [7]:
df_2016_xy = df_2016.iloc[:,:56].drop(['Dealer ID',"No. of Employees",\
                                       'registed 2016','JS 17.Column3','JS 18.Column3',\
                                       'JS 19.Column3','NON USER 17.Column3','NON USER 18.Column3',\
                                       'Velocity 17.Column3','centurty 18.Column3'],axis=1)

In [8]:
df_2016_xy.shape

(1473, 46)

In [24]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score

In [25]:
x = df_2016_xy.iloc[:,:45]
y = df_2016_xy.iloc[:,45].astype(int)

In [32]:
# scaler = StandardScaler()
# x_std = scaler.fit_transform(x)
def randomForest(x_std, y, test = 0.3,n_estimators_base=50, grid_cv_num = 5):
    x_train, x_test, y_train, y_test = train_test_split(x_std, y, test_size = test)
    rf = RandomForestClassifier(n_estimators=n_estimators_base)
    n_estimators_range = [150,175,200,250,300]
    param_grid = [{'n_estimators':n_estimators_range}]
    grid = GridSearchCV(rf,param_grid,cv=grid_cv_num,n_jobs=-1,verbose=1)
    clf = grid.fit(x_train,y_train)
    print("Best: %f using %s" % (clf.best_score_,clf.best_params_))
    pred = clf.predict(x_test)
    matrix = confusion_matrix(y_test, pred, labels=None, sample_weight=None)
    score = grid.score(x_test,y_test)
    print('accuracy:{}'.format(score))
    average_precision = average_precision_score(y_test, pred)
    print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
    F1_score=f1_score(y_test, pred, average='macro') 
    print('F1 score: {0:0.2f}'.format(
      F1_score))
    return pred,matrix

In [33]:
pred,matrix=randomForest(x,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.7s finished


Best: 0.585839 using {'n_estimators': 300}
accuracy:0.5656108597285068
Average precision-recall score: 0.58
F1 score: 0.56


In [13]:
confusion_matrix_1 = pd.DataFrame(matrix,columns=['pred_true','pred_false'],index=['real_true','real_false'])

In [14]:
confusion_matrix_1

,pred_true,pred_false
real_true,125,106
real_false,89,122
